In [0]:
# Define variables
storage_account_name = ""
container_name = "retail"
mount_point = "/mnt/retail_project"  # This is where the storage will be mounted in DBFS
access_key = ""

# Set the configuration
dbutils.fs.mount(
  source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
  mount_point = mount_point,
  extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": access_key}
)


True

In [0]:
# Check if mount is successful
display(dbutils.fs.ls("/mnt/retail_project/Bronze/transactions/"))


path,name,size,modificationTime
dbfs:/mnt/retail_project/Bronze/transactions/dbo.transactions.parquet,dbo.transactions.parquet,1757,1753361152000


In [0]:
df_transactions = spark.read.parquet("/mnt/retail_project/Bronze/transactions/")
df_products = spark.read.parquet("/mnt/retail_project/Bronze/products/")
df_stores = spark.read.parquet("/mnt/retail_project/Bronze/stores/")
df_customers = spark.read.parquet("/mnt/retail_project/Bronze/customers/gauravtiwari14/azure-data-engineer--retail-dataset/refs/heads/main/")

# df_transactions.printSchema()
df_customers.show(10)



+-----------+----------+---------+-------------------+----------+---------+-----------------+
|customer_id|first_name|last_name|              email|     phone|     city|registration_date|
+-----------+----------+---------+-------------------+----------+---------+-----------------+
|        101|      Ravi|    Yadav|user101@example.com|9887654321|    Delhi|       2023-09-14|
|        102|      Nina|    Joshi|user102@example.com|9876543210|   Mumbai|       2024-01-21|
|        103|     Sonal|   Sharma|user103@example.com|9865432109|Bangalore|       2023-07-10|
|        104|     Karan|    Patel|user104@example.com|9854321098|Hyderabad|       2024-02-05|
|        105|      Riya|    Singh|user105@example.com|9843210987|  Chennai|       2023-06-28|
|        106|      Ajay|   Mishra|user106@example.com|9832109876|     Pune|       2024-03-10|
|        107|     Priya|   Kapoor|user107@example.com|9821098765|Ahmedabad|       2023-05-12|
|        108|     Rahul|    Verma|user108@example.com|981098

In [0]:
from pyspark.sql.functions import col

df_transactions = df_transactions.select(
    col('transaction_id').cast('integer'),
    col('customer_id').cast('integer'),
    col('product_id').cast('integer'),
    col('store_id').cast('integer'),
    col('quantity').cast('integer'),
    col("transaction_date").cast('date')
    )

df_products = df_products.select(
    col('product_id').cast('integer'),
    col('product_name'),
    col('category'),
    col('price').cast('decimal(10,2)')
)

df_stores = df_stores.select(
    col('store_id').cast('integer'),
    col('store_name'),
    col('location')
)

df_customers = df_customers.select(
    col('customer_id').cast('integer'),
    col('first_name'),
    col('last_name'),
    col('email'),
    col('phone'),
    col('city'),
    col('registration_date').cast('date')
)

df_customers = df_customers.select(
    'customer_id', 'first_name', 'last_name', 'email', 'phone', 'city', 'registration_date'
).dropDuplicates(['customer_id']
)

In [0]:
df_transactions.show(5)


+--------------+-----------+----------+--------+--------+----------------+
|transaction_id|customer_id|product_id|store_id|quantity|transaction_date|
+--------------+-----------+----------+--------+--------+----------------+
|             1|        127|         8|       4|       4|      2025-03-31|
|             2|        105|         3|       4|       5|      2024-11-12|
|             3|        116|         2|       2|       3|      2025-05-01|
|             4|        120|         8|       1|       1|      2024-11-02|
|             5|        105|         5|       2|       1|      2025-03-17|
+--------------+-----------+----------+--------+--------+----------------+
only showing top 5 rows


In [0]:
# Join all Data
df_silver = df_transactions.join(df_customers,"customer_id").join(df_products,"product_id").join(df_stores,"store_id").withColumn('total_amount', col('quantity') * col('price'))


In [0]:
df_silver.show(5)

+--------+----------+-----------+--------------+--------+----------------+----------+---------+-------------------+----------+---------+-----------------+--------------+-----------+-------+-------------------+--------+------------+
|store_id|product_id|customer_id|transaction_id|quantity|transaction_date|first_name|last_name|              email|     phone|     city|registration_date|  product_name|   category|  price|         store_name|location|total_amount|
+--------+----------+-----------+--------------+--------+----------------+----------+---------+-------------------+----------+---------+-----------------+--------------+-----------+-------+-------------------+--------+------------+
|       1|         5|        108|            12|       4|      2025-05-26|     Rahul|    Verma|user108@example.com|9810987654|  Kolkata|       2023-08-19|  Notebook Set| Stationery| 149.00|    City Mall Store|  Mumbai|      596.00|
|       5|         7|        101|            28|       3|      2024-11-1

In [0]:
silver_path = "/mnt/retail_project/Silver/"

df_silver.write.mode("overwrite").format('delta').save(silver_path)

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS retail_silver_cleaned
    USING DELTA
    LOCATION '{silver_path}'
""")


DataFrame[]

In [0]:
%sql SELECT * FROM retail_silver_cleaned

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,phone,city,registration_date,product_name,category,price,store_name,location,total_amount
1,5,108,12,4,2025-05-26,Rahul,Verma,user108@example.com,9810987654,Kolkata,2023-08-19,Notebook Set,Stationery,149.00,City Mall Store,Mumbai,596.00
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,9887654321,Delhi,2023-09-14,Smartwatch,Electronics,4999.00,Mega Plaza,Chennai,14997.00
2,6,126,26,2,2024-09-21,Bhavna,Nair,user126@example.com,9632109876,Mumbai,2023-07-17,Water Bottle,Fitness,299.00,High Street Store,Delhi,598.00
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,9865432109,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.97
4,9,122,23,2,2025-04-30,Tina,Kapoor,user122@example.com,9676543210,Pune,2023-09-20,Dumbbell Set,Fitness,1999.00,Downtown Mini Store,Pune,3998.00
4,1,120,14,5,2024-07-17,Alka,Mishra,user120@example.com,9698765432,Hyderabad,2023-12-01,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,3999.95
1,8,127,27,1,2024-10-10,Rohit,Mehta,user127@example.com,9621098765,Bangalore,2023-08-30,Desk Organizer,Accessories,399.00,City Mall Store,Mumbai,399.00
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,9821098765,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.00,Tech World Outlet,Bangalore,7996.00
1,5,130,20,2,2024-07-30,Tanvi,Kapoor,user130@example.com,9598765432,Pune,2023-12-22,Notebook Set,Stationery,149.00,City Mall Store,Mumbai,298.00
3,1,102,11,2,2024-08-11,Nina,Joshi,user102@example.com,9876543210,Mumbai,2024-01-21,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,1599.98


In [0]:
silver_df = spark.read.format('delta').load(silver_path)


In [0]:
from pyspark.sql.functions import sum, avg, countDistinct

gold_df = silver_df.groupBy(
    'transaction_date',
    'product_id',
    'product_name',
    'category',
    'store_id',
    'store_name',
    'location'
).agg(
    sum('quantity').alias('total_quantity'),
    sum('total_amount').alias('total_amount'),
    countDistinct('transaction_id').alias('total_transactions'),
    avg('total_amount').alias('avg_transaction_value')
)


In [0]:
gold_df.show(5)

+----------------+----------+--------------+-----------+--------+-------------------+---------+--------------+------------+------------------+---------------------+
|transaction_date|product_id|  product_name|   category|store_id|         store_name| location|total_quantity|total_amount|total_transactions|avg_transaction_value|
+----------------+----------+--------------+-----------+--------+-------------------+---------+--------------+------------+------------------+---------------------+
|      2024-11-02|         8|Desk Organizer|Accessories|       1|    City Mall Store|   Mumbai|             1|      399.00|                 1|           399.000000|
|      2024-08-11|         1|Wireless Mouse|Electronics|       3|  Tech World Outlet|Bangalore|             2|     1599.98|                 1|          1599.980000|
|      2024-12-13|         8|Desk Organizer|Accessories|       4|Downtown Mini Store|     Pune|             5|     1995.00|                 1|          1995.000000|
|      202

In [0]:
gold_path = "/mnt/retail_project/Gold/"

gold_df.write.mode("overwrite").format('delta').save(gold_path)

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS retail_gold_sales_summary
    USING DELTA
    LOCATION '{gold_path}'
""")

DataFrame[]

In [0]:
%sql select * from retail_gold_sales_summary

transaction_date,product_id,product_name,category,store_id,store_name,location,total_quantity,total_amount,total_transactions,avg_transaction_value
2024-11-02,8,Desk Organizer,Accessories,1,City Mall Store,Mumbai,1,399.00,1,399.000000
2024-08-11,1,Wireless Mouse,Electronics,3,Tech World Outlet,Bangalore,2,1599.98,1,1599.980000
2024-12-13,8,Desk Organizer,Accessories,4,Downtown Mini Store,Pune,5,1995.00,1,1995.000000
2025-05-04,3,Yoga Mat,Fitness,3,Tech World Outlet,Bangalore,4,1996.00,1,1996.000000
2025-05-26,5,Notebook Set,Stationery,1,City Mall Store,Mumbai,4,596.00,1,596.000000
2024-07-14,1,Wireless Mouse,Electronics,5,Mega Plaza,Chennai,1,799.99,1,799.990000
2024-07-17,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,5,3999.95,1,3999.950000
2024-09-05,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,3,2399.97,1,2399.970000
2025-06-03,9,Dumbbell Set,Fitness,4,Downtown Mini Store,Pune,2,3998.00,1,3998.000000
2024-08-27,2,Bluetooth Speaker,Electronics,2,High Street Store,Delhi,5,6497.45,1,6497.450000
